In [ ]:
# import os
# import shutil
# structures_dir = "/home/mjustyna/data/Struktury/"
# seed = 0

# grail_preds_path = f"/home/mjustyna/RNA-GNN/samples/glowing-terrain-25-eval-clean-seed={seed}/800/"
# grail_preds = os.listdir(grail_preds_path)
# grail_preds = [f for f in grail_preds if f.endswith("_AA.pdb")]
# pdb_name = dict([(f.split("_")[0], f) for f in grail_preds])

# for pdb, name in pdb_name.items():
#     p = os.path.join(structures_dir, pdb, 'RNAgrail')
#     os.makedirs(p, exist_ok=True)
#     out_name = name.replace("_AA.pdb", f"_AA{seed}.pdb")
#     shutil.copy(grail_preds_path + name, p + '/' + out_name)
#     print(pdb, name)

In [ ]:
import os

structures_dir = "/home/mjustyna/data/Struktury/"
targets_dir = "/home/mjustyna/data/eval_examples_pdb/clean"
dirs = os.listdir(structures_dir)
targets = os.listdir(targets_dir)
targets_dict = dict([(x.split(".")[0], x) for x in targets])
targets_dict

In [ ]:
from Bio.PDB import PDBParser
from Bio.PDB import Superimposer
from Bio import BiopythonWarning
from pymol import cmd
import barnaba as bb
import warnings
import math
warnings.simplefilter('ignore', BiopythonWarning)
from rnapolis.annotator import extract_secondary_structure
from rnapolis.parser import read_3d_structure
import numpy as np

def align_biopython(pred_path, pdb_pred, targets_path, pdb_gt):
    parser = PDBParser()
    ref_structure = parser.get_structure('ref', f"{targets_path}/{pdb_gt}")
    ref_model = ref_structure[0]
    ref_atoms = [atom for atom in ref_model.get_atoms()]
    
    pred_structure = parser.get_structure('structure', f"{pred_path}/{pdb_pred}")
    model = pred_structure[0]
    atoms = [atom for atom in model.get_atoms()]
    sup = Superimposer()
    rmsd = np.nan
    try:
        sup.set_atoms(ref_atoms, atoms) # if there is a missing P atom in some chain, then is should be removed from prediction to do superposition
        sup.apply(model.get_atoms())
        rmsd = round(sup.rms, 3)
    except:
        print(f"Skipping {pdb_pred} as the superimposition failed")
        
    return rmsd
    
def align_pymol(pred_path, pdb_pred, targets_path, pdb_gt):
    cmd.reinitialize()
    cmd.load(f"{targets_path}/{pdb_gt}", "ref")
    cmd.load(f"{pred_path}/{pdb_pred}", "structure")
    rms = cmd.align("ref", "structure")[0]
    cmd.delete("ref")
    cmd.delete("structure")
    return round(rms, 3)

def extract_2d_structure(pdb_path):
    with open(pdb_path, 'r') as f:
        structure3d = read_3d_structure(f, 1)
        structure2d = extract_secondary_structure(structure3d, 1)
    s2d = structure2d.dotBracket.split('>strand')
    if len(s2d) > 1:
        s2d = [s.strip().split('\n')[-1] for s in s2d]
        return "".join(s2d)
    else:
        return structure2d.dotBracket.split('\n')[-1]

def get_inf(pred_path, pdb_pred, targets_path, pdb_gt):
    s_gt = extract_2d_structure(f"{targets_path}/{pdb_gt}")
    s_pred = extract_2d_structure(f"{pred_path}/{pdb_pred}")
    assert len(s_pred) == len(s_gt), ValueError(f"Length of the predicted and ground truth sequences should be the same, but are {len(s_pred)} and {len(s_gt)}")
    s_pred = [1 if c != '.' else 0 for c in s_pred]
    s_gt = [1 if c != '.' else 0 for c in s_gt]
    tp = sum([1 for i in range(len(s_pred)) if s_pred[i] == 1 and s_gt[i] == 1])
    fp = sum([1 for i in range(len(s_pred)) if s_pred[i] == 1 and s_gt[i] == 0])
    fn = sum([1 for i in range(len(s_pred)) if s_pred[i] == 0 and s_gt[i] == 1])
    ppv = tp / (tp + fp) if tp + fp > 0 else 0
    sty = tp / (tp + fn) if tp + fn > 0 else 0
    inf = math.sqrt(ppv * sty)
    return inf


In [ ]:
import numpy as np
struct_dict_rmsd = {}
struct_dict_inf = {}
struct_dict_ermsd = {}
substitute_dict = {
    'ALPHAFOLD': 'ALPHAFOLD3',
    'RHOFOLD': 'RHOFOLD_SEQ',
    'RHOFOL_SEQ': 'RHOFOLD_SEQ',
    'RHOFOLD_RELAXED': 'RHOFOLD_SEQ',
    'TRROSETTA': 'TRROSETTARNA',
    'TRROSETTA_2D': 'TRROSETTARNA_2D',
    
}

for d in dirs:
    models = os.listdir(structures_dir + d)

    model_rmsd_dict = {}
    model_inf_dict = {}
    model_ermsd_dict = {}
    for m in models:
        print(d, m)
        if os.path.isdir(os.path.join(structures_dir, d, m)) == False:
            continue
        pdbs = os.listdir(os.path.join(structures_dir, d, m))
        pdbs = [x for x in pdbs if x.endswith(".pdb") or x.endswith(".cif")]
        if len(pdbs) == 0:
            continue
        for pdb in pdbs:
            rmsd = align_pymol(os.path.join(structures_dir, d, m), pdb, targets_dir, targets_dict[d])
            try:
                inf = get_inf(os.path.join(structures_dir, d, m), pdb, targets_dir, targets_dict[d])
                ermsd = bb.ermsd(f"{targets_dir}/{targets_dict[d]}", f"{os.path.join(structures_dir, d, m)}/{pdb}")[0]
                print(inf, ermsd)
            except AssertionError as e:
                print(e)
                inf = np.nan
                ermsd = np.nan
            
            dict_name = m.upper()
            if dict_name in substitute_dict:
                dict_name = substitute_dict[m.upper()]
            rmsd_name = dict_name+"_RMSD"
            inf_name = dict_name+"_INF"
            ermsd_name = dict_name+"_ERMSD"
            if rmsd_name not in model_rmsd_dict:
                model_rmsd_dict[rmsd_name] = []
                model_inf_dict[inf_name] = []
                model_ermsd_dict[ermsd_name] = []
            
            model_rmsd_dict[rmsd_name].append(rmsd)
            model_inf_dict[inf_name].append(inf)
            model_ermsd_dict[ermsd_name].append(ermsd)

    struct_dict_rmsd[d] = model_rmsd_dict
    struct_dict_inf[d] = model_inf_dict
    struct_dict_ermsd[d] = model_ermsd_dict
    print(model_rmsd_dict)

In [ ]:
import pandas as pd
import numpy as np

def make_dataframe(struct_dict, key):
    df = pd.DataFrame(struct_dict)
    df = df.T
    # compute mean in all cells
    df = df.map(lambda x: np.mean(x))
    # drop rows where alphafold is missing
    df = df.dropna(subset=['ALPHAFOLD3_'+key])
    # sort by number of values in each record (complete rows are first). Present in alphabetical 
    a = df.count(axis=1).sort_values(ascending=False)
    df = df.loc[a.index]
    return df
df_rmsd = make_dataframe(struct_dict_rmsd, key="RMSD")
df_inf = make_dataframe(struct_dict_inf, key="INF")
df_ermsd = make_dataframe(struct_dict_ermsd, key="ERMSD")
df_rmsd.to_csv("rmsd_pymol.csv")
df_inf.to_csv("inf.csv")
df_ermsd.to_csv("ermsd.csv")

In [ ]:
df_all = pd.concat([df_rmsd, df_inf, df_ermsd], axis=1)
# change column order to: RNAGRAIL_RMSD, RNAGRAIL_INF, RNAGRAIL_ERMSD, ALPHAFOLD3_RMSD, ALPHAFOLD3_INF, ALPHAFOLD3_ERMSD, ...
df_all = df_all.reindex(['RNAGRAIL_RMSD', 'RNAGRAIL_INF', 'RNAGRAIL_ERMSD', 'ALPHAFOLD3_RMSD', 'ALPHAFOLD3_INF', 'ALPHAFOLD3_ERMSD', 'RHOFOLD_SEQ_RMSD', 'RHOFOLD_SEQ_INF', 'RHOFOLD_SEQ_ERMSD', 'RHOFOLD_MSA_RMSD', 'RHOFOLD_MSA_INF', 'RHOFOLD_MSA_ERMSD','TRROSETTARNA_RMSD', 'TRROSETTARNA_INF', 'TRROSETTARNA_ERMSD','TRROSETTARNA_2D_RMSD', 'TRROSETTARNA_2D_INF', 'TRROSETTARNA_2D_ERMSD', 'DEEPFOLDRNA_RMSD', 'DEEPFOLDRNA_INF', 'DEEPFOLDRNA_ERMSD'], axis=1)
df_all.to_csv("rmsd_inf_ermsd_pymol.csv")

In [ ]:

df_all

In [ ]:
bb.ermsd("/home/mjustyna/data/eval_examples_pdb/clean/8HZE.pdb", "/home/mjustyna/data/Struktury/8HZE/RNAgrail/8HZE_AA2.pdb")[0]

In [ ]:
# Remove residues incomplete in the ground truth
# rna_pdb_tools.py --delete A:1 fold_pdb_rna_8fza_chain_a_model_4_fCIF.pdb > model4.pdb

# convert cif to pdb
# rna_pdb_tools.py --cif2pdb fold_pdb_rna_8fza_chain_a_model_4_fCIF.pdb > model4.pdb